In [48]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

# Function to remove stop words from a text
def remove_stop_words(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return " ".join(filtered_text)

def remove_punkt(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.isalnum()]
    return " ".join(filtered_text)

[nltk_data] Downloading package stopwords to /Users/dung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/dung/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Rouge score

In [52]:
from rouge_score import rouge_scorer
import re

def clean_to_compare(text):
    text = remove_stop_words(text)
    text = remove_punkt(text)

    # remove actions
    text = re.sub(r'^(Convert|Make|Reframe|Adapt|Change|Modify|Transcribe|Translate|Transform|Rephrase|Rewrite|Revise|Edit|Alter|Amend|Adjust|Redraft|Reform|Rescript|Revamp|Vary|Convert|Turn|Transmute|Metamorphose|Metamorphize|Transfigure|Reconstruct|Edit|Add)', '', text)

    # noun
    text = re.sub(r'text|essay|paragraph|passage', '', text)
    text = text.strip()

    return text

scorer_for_rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def do_rouge_score(text1, text2):
    text1, text2 = list(map(clean_to_compare, [text1, text2]))
    print([text1, text2])
    scores = scorer_for_rouge.score(text1, text2)
    return scores

# Replace these with your sentences
sentences = [
    "Convert this into a sea shanty",
    "Make this a sea shanty",
    "Reframe this text to be a sea shanty",
    "Edit this passage for better flow and coherence.",
    "Transcribe this paragraph into a modern epigram."
]

for s in sentences[1:]:
    scores = do_rouge_score(sentences[0], s)
    print(scores)

['sea shanty', 'sea shanty']
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
['sea shanty', 'sea shanty']
{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rouge2': Score(precision=1.0, recall=1.0, fmeasure=1.0), 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}
['sea shanty', 'better flow coherence']
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}
['sea shanty', 'modern epigram']
{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


T5 scoreing

In [21]:
import torch
from sentence_transformers import SentenceTransformer

t5base_model = None

def get_t5base_model():
    global t5base_model
    if t5base_model is None:
        t5base_model = SentenceTransformer('sentence-t5-base')
        if torch.cuda.is_available():
            t5base_model = t5base_model.to(torch.device("cuda"))
    return t5base_model

# Function to calculate sharpened cosine similarity
def sharpened_cosine_similarity(vec1, vec2, exponent=3):
    cosine_similarity = torch.nn.functional.cosine_similarity(vec1, vec2, dim=0)
    return cosine_similarity ** exponent

#provides similarity scores of a test_phrase against an array of phrases
def compare_phrases(test_phrase, phrases, model=None):
    if model is None:
        model = get_t5base_model()

    scores = []
    test_embedding = model.encode(test_phrase, convert_to_tensor=True, show_progress_bar=False)

    for phrase in phrases:
        compare_embedding = model.encode(phrase, convert_to_tensor=True, show_progress_bar=False)
        score = sharpened_cosine_similarity(test_embedding, compare_embedding).item()
        scores.append(score)

    return scores

In [37]:
import numpy as np

target = "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."
target = "Refine the following passage by emulating the writing style of [insert desired style here], with a focus on enhancing its clarity, elegance, and overall impact. Preserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style.Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style."

targets = [
    "Please improve the following text using the writing style of, maintaining the original meaning",
    "Improve the following text using the writing style of, maintaining the original meaning",
    "Please improve the following text using the writing style of",
    "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.",
    "Please enhance the clarity, elegance, and impact of the following text by adopting the writing style of",
    "Please enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.",
    "Please refine the following passage by emulating the writing style of, with a focus on enhancing its clarity, elegance, and overall impact.",
    "Please rreserve the essence and original meaning of the text, while meticulously adjusting its tone, vocabulary, and stylistic elements to resonate with the chosen style",
    "Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style",
    "Please enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.",
]

texts = [
    "Reframe this text to be more engaging for a workshop.",
    "Make this text more suitable for a professional setting.",
    "Adapt this text to fit the format of a TED Talk script.",
    "Change the structure of this paragraph to be more engaging.",
    "Adjust the tone of this text to be more educational.",
    "Modify this text to sound lyrical as in a song.",
    "Transcribe this paragraph into a modern epigram.",
    "Rewrite this passage as a text message conversation.",
    "Paraphrase this passage to make it sound like a story.",
    "Transform this text to be in the style of a screenplay.",
    "Reconstruct this text as a piece of spoken word poetry.",
    "Revise the original text into a dialogue between two people.",
    "Rewrite this text as if it were spoken word poetry.",
    "Reframe this text for a professional audience.",
    "Reconstruct this text to be more contemplative.",
    "Revamp this text to sound like a marketing pitch.",
    "Add detail to this text to enhance the reader's understanding.",
    "Edit this passage for better flow and coherence.",
    "Revamp this text for a poetry reading.",
    "Rewrite this text to make it sound like an interview.",
]

# target = remove_stop_words(target)
# texts = [remove_stop_words(text) for text in texts]

for target in targets:
    scores = compare_phrases(target, texts)
    indices = np.argsort(scores)[::-1]

    print(f"\n=== Score: {np.mean(scores)}, Target: {target}\n")

    # for t, s in zip(texts, scores):
    #     print(f"{t} => {s:.4f}")

    for i in indices:
        print(f"{texts[i]} => {scores[i]:.4f}")



=== Score: 0.6523946566241128, Target: Please improve the following text using the writing style of, maintaining the original meaning

Edit this passage for better flow and coherence. => 0.7442
Change the structure of this paragraph to be more engaging. => 0.7419
Add detail to this text to enhance the reader's understanding. => 0.7122
Make this text more suitable for a professional setting. => 0.7075
Adjust the tone of this text to be more educational. => 0.7019
Revamp this text for a poetry reading. => 0.6987
Reconstruct this text to be more contemplative. => 0.6899
Revise the original text into a dialogue between two people. => 0.6760
Reframe this text to be more engaging for a workshop. => 0.6729
Modify this text to sound lyrical as in a song. => 0.6603
Transform this text to be in the style of a screenplay. => 0.6526
Transcribe this paragraph into a modern epigram. => 0.6514
Rewrite this text as if it were spoken word poetry. => 0.6497
Reframe this text for a professional audience

# model playground